In [ ]:
import os
import sys

import requests

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.ingestion.auth import strava_auth  # noqa: E402

In [ ]:
athlete_url = 'https://www.strava.com/api/v3/athlete'
access_token = strava_auth.get_access_token()
headers = {'Authorization': f'Bearer {access_token}'}

# General athlete information

In [ ]:
def get_debug(url, headers, **kwargs):
    try:
        r = requests.get(url, headers=headers, timeout=10, **kwargs)
        r.raise_for_status()
        print('OK', r.status_code)
        return r
    except requests.HTTPError as e:
        r = e.response
        hdrs = {
            k: r.headers.get(k)
            for k in [
                'X-RateLimit-Limit',
                'X-RateLimit-Usage',
                'X-ReadRateLimit-Limit',
                'X-ReadRateLimit-Usage',
                'Retry-After',
                'Date',
            ]
        }
        print('HTTPError:', r.status_code)
        print('Headers:', hdrs)
        print('Body (first 300):', r.text[:300])
        raise


athlete_url = 'https://www.strava.com/api/v3/athlete'
resp = get_debug(athlete_url, headers=headers)
data = resp.json()

In [ ]:
data

In [ ]:
athlete_id = data['id']
print(f'Athlete ID: {athlete_id}')

# Athlete stats

In [ ]:
athlete_stats_url = f'https://www.strava.com/api/v3/athletes/{athlete_id}/stats'
r = requests.get(athlete_stats_url, headers=headers, timeout=10)
r.raise_for_status()

In [ ]:
athlete_stats = r.json()
athlete_stats

# Gears

- To get the id of a gear one have to check it manually or iterate over all activities and create a list
- Since they don't change that often, it makes sense to manually create a list with all gear ids

In [ ]:
gear_id = 'g20984891'
gear_2 = 'b16370167'

In [ ]:
gear_url = f'https://www.strava.com/api/v3/gear/{gear_2}'
r = requests.get(gear_url, headers=headers, timeout=10)
gear = r.json()
gear

# Activity Details

## Streams

In [ ]:
activity_id = '16222086377'

In [ ]:
stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams?heartrate'
r = requests.get(stream_url, headers=headers, timeout=10)
stream = r.json()

In [ ]:
import requests

activity_id = '16222086377'
stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams'

params = {
    'keys': 'time,heartrate,altitude,velocity_smooth',  # or use 'types'
    'key_by_type': 'true',
}

r = requests.get(stream_url, headers=headers, params=params, timeout=10)
r.raise_for_status()
stream = r.json()

print(
    stream.keys()
)  # e.g. dict_keys(['time', 'heartrate', 'altitude', 'velocity_smooth'])

In [ ]:
import pandas as pd

print(pd.__version__)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import requests

stream_url = f'https://www.strava.com/api/v3/activities/{activity_id}/streams'

params = {'keys': 'time,heartrate', 'key_by_type': 'true'}

r = requests.get(stream_url, headers=headers, params=params, timeout=10)
r.raise_for_status()
stream = r.json()

# --- Convert to DataFrame ---
df = pd.DataFrame({
    'time_s': stream['time']['data'],
    'heart_rate': stream['heartrate']['data'],
})

# Optional: convert seconds to minutes
df['time_min'] = df['time_s'] / 60

# --- Plot ---
plt.figure(figsize=(10, 5))
plt.plot(df['time_min'], df['heart_rate'], linewidth=1.5)
plt.title(f'Heart Rate Stream for Activity {activity_id}')
plt.xlabel('Time (minutes)')
plt.ylabel('Heart Rate (bpm)')
plt.grid(True)
plt.tight_layout()
plt.show()

## Comments

In [ ]:
comment_url = f'https://www.strava.com/api/v3/activities/{activity_id}/comments'
r = requests.get(comment_url, headers=headers, timeout=10)
r.raise_for_status()
comments = r.json()
comments

## Kudos

In [ ]:
kudos_url = f'https://www.strava.com/api/v3/activities/{activity_id}/kudos'
r = requests.get(kudos_url, headers=headers, timeout=10)
r.raise_for_status()
kudos = r.json()
kudos

## Laps

In [ ]:
laps_url = f'https://www.strava.com/api/v3/activities/{activity_id}/laps'
r = requests.get(laps_url, headers=headers, timeout=10)
r.raise_for_status()
laps = r.json()
laps

## Zones

In [ ]:
zones_url = f'https://www.strava.com/api/v3/activities/{activity_id}/zones'
r = requests.get(zones_url, headers=headers, timeout=10)
r.raise_for_status()
zones = r.json()
zones